In [1]:
import numpy as np
import MDAnalysis as mda 
import nglview
import matplotlib.pyplot as plt

# Data prep

In [2]:
#Should only need to run through this section once to prepare the data for training a VAE
#Load topology and trajectory into MDAnalysis "Universe"
uni = mda.Universe('sim_files/alanine-dipeptide.prmtop', 'sim_files/ala_dipeptide.nc')

trajtimes = np.arange(0, uni.trajectory.totaltime+1e-06, uni.trajectory.dt)

for a in uni.atoms:
    print(a)

<Atom 1: HH31 of type HC of resname ACE, resid 1 and segid SYSTEM>
<Atom 2: CH3 of type CT of resname ACE, resid 1 and segid SYSTEM>
<Atom 3: HH32 of type HC of resname ACE, resid 1 and segid SYSTEM>
<Atom 4: HH33 of type HC of resname ACE, resid 1 and segid SYSTEM>
<Atom 5: C of type C of resname ACE, resid 1 and segid SYSTEM>
<Atom 6: O of type O of resname ACE, resid 1 and segid SYSTEM>
<Atom 7: N of type N of resname ALA, resid 2 and segid SYSTEM>
<Atom 8: H of type H of resname ALA, resid 2 and segid SYSTEM>
<Atom 9: CA of type CT of resname ALA, resid 2 and segid SYSTEM>
<Atom 10: HA of type H1 of resname ALA, resid 2 and segid SYSTEM>
<Atom 11: CB of type CT of resname ALA, resid 2 and segid SYSTEM>
<Atom 12: HB1 of type HC of resname ALA, resid 2 and segid SYSTEM>
<Atom 13: HB2 of type HC of resname ALA, resid 2 and segid SYSTEM>
<Atom 14: HB3 of type HC of resname ALA, resid 2 and segid SYSTEM>
<Atom 15: C of type C of resname ALA, resid 2 and segid SYSTEM>
<Atom 16: O of type

/Users/jim2/bin/MDAnalysis/mdanalysis/package/MDAnalysis/topology/TOPParser.py:288: UserWarning: ATOMIC_NUMBER record not found, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn(msg)


In [3]:
from MDAnalysis import transformations

#Want to try two coordinate systems
#For both will use C-alpha as a reference (and the first frame for rotations)
#For first, remove translation of reference atom, then align via rigid rotation matrix
#For second, take first set of coordinates and convert to bond-angle-torsion
#Will also have third set of Cartesian, but with no hydrogens, just heavy atoms

#Remove rotation and COM translation
uni.trajectory.add_transformations(transformations.fit_rot_trans(uni.select_atoms('all'), uni.select_atoms('all')))


In [4]:
#Can watch the trajectory with nglview!
#If have already applied transformation, will not move very much (due to alignment)
view = nglview.show_mdanalysis(uni)
#view.player.parameters={'step':10}
view

NGLWidget(max_frame=999999)

In [5]:
from MDAnalysis.analysis import bat

#Now perform bond-angle-torsion conversion
#Note we only take every 10th frame, so every 10 ps
bat_analysis = bat.BAT(uni.select_atoms('all'))
bat_analysis.run(step=10)

bat_coords = bat_analysis.bat
print(bat_coords.shape)

(100000, 66)


In [6]:
bat_analysis.save('ala_dipeptide_BAT')

In [ ]:
#Randomly shuffle so batches are balanced
bat_coords = np.load('ala_dipeptide_BAT.npy')
np.random.shuffle(bat_coords)
np.save('ala_dipeptide_BAT', bat_coords)

In [14]:
#Want to get all XYZ coordinates and save flattened version for easy training
#Note we only take every 10th frame, so every 10 ps
xyz_coords = np.zeros(bat_coords.shape)

#And also pull out only heavy atoms, removing hydrogens
h_inds = [a.index for a in uni.select_atoms('name H*')] #[0, 2, 3, 7, 9, 11, 12, 13, 17, 19, 20, 21]
h_mask = np.ones(bat_coords.shape[1]//3, dtype=bool)
h_mask[h_inds] = False
xyz_coords_heavy = np.zeros((xyz_coords.shape[0], xyz_coords.shape[1]-(3*len(h_inds))))

for t, frame in enumerate(uni.trajectory[::10]):
    xyz_coords[t, :] = frame.positions.flatten()
    xyz_coords_heavy[t, :] = frame.positions[h_mask, :].flatten()

print(xyz_coords.shape)
print(xyz_coords_heavy.shape)

(100000, 66)
(100000, 30)


In [15]:
np.random.shuffle(xyz_coords)
np.save('ala_dipeptide_XYZ', xyz_coords)

In [16]:
np.random.shuffle(xyz_coords_heavy)
np.save('ala_dipeptide_XYZheavy', xyz_coords_heavy)